## Notebook for making predictions with deleted constant y samples


In [152]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [153]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [154]:
X_A_train = dm.data_A.iloc[:, 2:-1]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:-1]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:-1]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

Y_A_train


0         0.00
1         0.00
2         0.00
3         0.00
4        19.36
         ...  
23219     9.02
23220     0.00
23221     0.00
23222     0.00
23223     0.00
Name: pv_measurement, Length: 23204, dtype: float64

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [155]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [156]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

Learning rate set to 0.067288
0:	learn: 1246.5294425	total: 16ms	remaining: 16s
1:	learn: 1184.7353723	total: 23.4ms	remaining: 11.7s
2:	learn: 1128.3257621	total: 30.9ms	remaining: 10.3s
3:	learn: 1076.3634112	total: 38.1ms	remaining: 9.49s
4:	learn: 1028.6386699	total: 45.5ms	remaining: 9.06s
5:	learn: 982.8716471	total: 52.6ms	remaining: 8.71s
6:	learn: 942.5876631	total: 59.4ms	remaining: 8.43s
7:	learn: 905.7632550	total: 66.5ms	remaining: 8.24s
8:	learn: 871.8700875	total: 73.5ms	remaining: 8.09s
9:	learn: 840.9379182	total: 80ms	remaining: 7.92s
10:	learn: 811.5032855	total: 86.4ms	remaining: 7.77s
11:	learn: 785.2869715	total: 92.8ms	remaining: 7.64s
12:	learn: 761.4912852	total: 99.3ms	remaining: 7.54s
13:	learn: 739.5073646	total: 106ms	remaining: 7.45s
14:	learn: 718.8876715	total: 112ms	remaining: 7.34s
15:	learn: 701.0934079	total: 118ms	remaining: 7.25s
16:	learn: 684.7085900	total: 124ms	remaining: 7.17s
17:	learn: 670.1799626	total: 130ms	remaining: 7.08s
18:	learn: 657

## Preds

In [157]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

In [158]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/catboost_donate_17.csv")

df.shape

(2160, 1)